In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd

import sys
sys.path.append("../")
from bargains_logit import *

In [2]:
def clip_p(p1, p2,  cost, wtp):
    s1,s2 = calc_shares(p1,p2,cost,wtp)
    p1_clip = np.clip(p1,0,wtp[0]-s1*cost)
    p2_clip = np.clip(p2,0,wtp[1]-s2*cost)
    return p1, p2
    

def calc_profits_price_shares_m2(phi1,phi2,cost,wtp,mc):
    mc1,mc2 = mc
    
    clip_p1 = lambda p : clip_p(p[0],p[1],cost,wtp)
    
    pi = lambda p : -1*calc_shares(clip_p1(p)[0], clip_p1(p)[1],cost, wtp)[0]*(clip_p1(p)[0]-phi1 )\
                    + -1*calc_shares(clip_p1(p)[0], clip_p1(p)[1], cost, wtp)[1]*(clip_p1(p)[1]-phi2 )

    p = minimize(pi,[30,30]).x
    p1,p2 = clip_p1(p)[0], clip_p1(p)[1]
    s1,s2 = calc_shares(p1, p2,  cost, wtp)
    return  p1,p2, s1,s2, s1*(p1-phi1-mc[0]), s2*(p2-phi2-mc[1])


calc_profits_price_shares_m2(30,30,5,np.array([25,25]),np.array([0,0]))

(36.08857090861948,
 36.08857090861948,
 0.08939499474696039,
 0.08939499474696039,
 0.5442877643925343,
 0.5442877643925343)

In [3]:
#arbitrary outside option...
def nash_in_nash_m2(phi1, phi2, cost, wtp, mc, beta=.5,active=False):
    p1,p2,s1,s2,pi1,pi2 = calc_profits_price_shares_m2(phi1,phi2,cost,wtp,mc)
    #print(p1,phi1,s1,s1*phi1,s1*(p1-phi1-mc[0]))
    obj = -1*(np.log(max(s1*phi1,1e-4))*(1-beta)  + np.log(s1*(p1-phi1-mc[0]))*beta)
    return obj


def bargain_helper_m2(phi1, phi2, cost, wtp, mc, beta=.5,active=False):
    """solve each firm 1s optimization holding phi 2 fixed"""
    result = minimize(nash_in_nash_m2, phi1, args=(phi2,cost, wtp, mc, beta, active),
                      method='Nelder-Mead', options={'disp': False})
    return result.x


def simult_bargain_m2(phi1, phi2, cost, wtp, mc, betas=[.5,.5],active=False):
    """solve the bargaining problems seperately and iterate for a solution"""       
    
    #loop variables, check on this...
    diff =  np.maximum(phi1,phi2)
    phi10,phi20 = 0,0
    maxiter = 10
    while maxiter >=0 and diff > 10e-7:
        #seems as though there is a contraction mapping here, need to think more about why
        phi1 = bargain_helper_m2(phi1, phi2, cost, wtp, mc, beta=betas[0] ,active=active)
        phi2 = bargain_helper_m2(phi2, phi1, cost, wtp, mc[::-1], beta=betas[1],active=active)
        
        #update loop variables
        diff = np.abs(np.maximum(phi1 - phi10,phi2-phi20))[0]
        phi10,phi20 = phi1,phi2
        maxiter = maxiter-1
        
    return phi1, phi2


COST = 5
WTP = np.array([25,25])
MC = np.array([0,0])


print(bargain_helper_m2(20, 20, COST, WTP, MC, beta=.5,active=False))
print(simult_bargain_m2(20, 20, COST, WTP, MC, betas=[.5,.5],active=False))
print(simult_bargain_m2(20, 20, COST, WTP, MC, betas=[.423,.203],active=False))

[8.21697998]
(array([4.29300199]), array([4.30111233]))
(array([5.39435739]), array([6.53425722]))


In [4]:
# Read in the necessary input data
param_data = pd.read_csv('../GMM_est/param_tables_logit_reimb_seq.csv')

# Initialize a results dataframe with the specified rows and an empty vector for the columns to be filled in
results = pd.DataFrame(index=['beta1', 'beta2', 'phi1', 'phi2', 'p1', 'p2'])

# Define the function to clean data
def clean_data(df):
    df = df[(df['s1']!=0) & (df['s1']!=1) & (df['phi1']>=0) & (df['phi2']>=0) & (np.abs(df['phi1'])<=100) & (np.abs(df['phi2'])<=100)]
    return df

# Initialize column 1
beta1, beta2 = 0.5, 0.5
seq_result = clean_data(pd.read_csv('../GMM_est/fake_data_wtp_logit_fixed2/seq_data_0.csv', index_col=0))
phi1, phi2, p1, p2 = seq_result[['phi1', 'phi2', 'p1', 'p2']].mean().to_list()
results['column1'] = [beta1, beta2, phi1, phi2, p1, p2]

# Prepare list of betas, WTP, and COST for columns 2-4
parameters_list = [
    ([0.5, 0.5], 31, 5),  # Column 2
    ([(param_data['beta1'][0] + param_data['beta2'][0]) / 2] * 2, param_data['wtp'][0], param_data['cost'][0]),  # Column 3
    ([(param_data['beta1'][1] + param_data['beta2'][1]) / 2] * 2, param_data['wtp'][1], param_data['cost'][1]),  # Column 4
]

# Loop through the parameters list and create columns 2-4
for i, (betas, WTP, COST) in enumerate(parameters_list):
    phis = simult_bargain_m2(20, 20, COST, [WTP,WTP], [0,0], betas=betas, active=True)
    phi1, phi2 = phis[0][0], phis[1][0]
    p1, p2, s1, s2, pi1, pi2 = calc_profits_price_shares_m2(phi1, phi2, COST, [WTP,WTP], [0,0])
    beta1, beta2 = betas
    results[f'column{i+2}'] = [beta1, beta2, phi1, phi2, p1, p2]

# Format the results to show only 3 significant digits
#results = results.applymap(lambda x: f"{x:.3g}")

# Save the results to a CSV
results.to_csv('merger_results.csv', index_label='parameter')

In [5]:
print(results)

         column1    column2    column3    column4
beta1   0.500000   0.500000   0.452526   0.349246
beta2   0.500000   0.500000   0.452526   0.349246
phi1   18.156926   4.430416   4.869382   5.855661
phi2   20.425130   4.425144   4.872121   5.864934
p1     27.032116  27.927511  28.039015  28.248405
p2     28.108806  27.922225  28.041514  28.257653
